<a href="https://colab.research.google.com/github/claudio1975/Medium-blog/blob/master/Medical_Q%26A_from_BoW_to_Agents/Medical_Q%26A_fromBoW_to_Agents_zephyr_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch transformers sentence-transformers faiss-cpu pypdf &> /dev/null


In [ ]:
!pip install -U langchain-huggingface &>/dev/null


In [ ]:
!pip install -q langchain langchain-community &> /dev/null


In [ ]:
!pip install ipywidgets &>/dev/null

In [ ]:
! pip install huggingface_hub[hf_xet] &> /dev/null

In [ ]:
! pip install -U "autogen[openai]" &>/dev/null

In [ ]:
import os
import pandas as pd
import numpy as np
import langchain as lc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain import LLMMathChain
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
import autogen
from autogen import AssistantAgent

In [ ]:
llm_config = {
    "model": "gpt-4o-mini",
    "api_key": ""
    }

In [ ]:
dataset = pd.read_parquet("hf://datasets/lavita/ChatDoctor-iCliniq/data/train-00000-of-00001-7f15f39e4c3a7ee9.parquet")
# Select relevant columns
selected_columns = ['input', 'answer_icliniq']
train_dataset = dataset[selected_columns].copy()

# Convert the DataFrame to a list of Document objects
documents = [
    Document(page_content=row['input'], metadata={'answer': row['answer_icliniq']})
    for _, row in train_dataset.iterrows()
]


In [ ]:
#-----------------
# Look up
#-----------------


In [ ]:
# Define function to retrieve answers
def get_answer_exact(query, df):
    result = train_dataset[train_dataset["input"] == query]["answer_icliniq"]
    return result.iloc[0] if not result.empty else "No exact match found"


In [ ]:
# Example query
query = "Hi doctor, What are the symptoms of diabetes?"
print(get_answer_exact(query, train_dataset))



No exact match found


In [ ]:
# Example search
query = "HI doctor, What are common symptoms of flu?"
print(get_answer_exact(query, train_dataset))



No exact match found


In [ ]:
# Example search
query= "Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (oral). Now, I am feeling the pain frequently in the right lower abdomen with mild bleeding and pain. It is less when I lie on bed. What to do?"
print(get_answer_exact(query, train_dataset))



Hello, Welcome to Chat Doctor forum. I have read through your question and understand your concerns. I think it is threatened abortion. You should take bed rest along with tablet Susten 200 mg twice daily (Progesterone) for two weeks and Mbryosafe sachet once daily (L-arginine). I think it will help you and give you better prognosis. Ultrasound is required after two weeks for evaluation.


In [ ]:
# Example search
query="Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigmentation caused by acne before. I only get acne on my cheeks. What may be the reason? I usually get cyst acne bumps on my cheeks. If acne clears, then it comes again. I have red pigmentation caused by acne. Can you recommend a product for acne and after acne has cleared, can you suggest a product for pigmentation? Can you recommend a product for controlling further breakouts? Also, I wanted to know which sunscreen and moisturizer to use?"
print(get_answer_exact(query, train_dataset))



Hi. Acne reduces in severity with aging. Use cleanser like Cetaphil to wash face. Apply Retino-A (Retinol) cream in the night, Clindamycin gel in the morning. Treat dan ChatDoctor.  You can use sunscreens like Shade (Avobenzone, Oxybenzone) lotion. Retino-A helps in pigmentation and to some extent scars. You can consider lasers later if scars persist. For more information consult a dermatologist online


In [ ]:
# Example search
query="Hello doctor,I am diabetic and hypertensive and have neuro problems. I want you to look at my MRI reports and advice me accordingly. I have attached the reports."
print(get_answer_exact(query, train_dataset))


Hello, Welcome to Chat Doctor forum. I have gone through the attachments (attachment removed to protect patient identity). The MRI (magnetic resonance imaging) report says that there are chronic ischemic changes suggestive of small vessel ischemia. These changes are usually seen in people with underlying diabetes and hypertension and represent longstanding nature of two conditions. The only way to prevent further progression of the changes is strict control of diabetes and hypertension. These changes in the brain will not lead to any disability. However, you should understand that you are at increased risk of stroke and hence strict control of diabetes and hypertension is essential. For more information consult a neurosurgeon online


In [ ]:
#---------
# BoW
#---------



In [ ]:
# Create BoW / TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_dataset["input"])  # Convert input column into vector form

# Function to find the closest matching input
def retrieve_answer_bow(query, df, vectorizer, X):
    query_vec = vectorizer.transform([query])  # Vectorize the query
    similarities = cosine_similarity(query_vec, X).flatten()  # Compute similarity
    best_match_idx = similarities.argmax()  # Get the closest match index
    return df.iloc[best_match_idx]["answer_icliniq"]


In [ ]:
# Example search
query = "Hi doctor, What are the symptoms of diabetes?"
answer = retrieve_answer_bow(query, train_dataset, vectorizer, X)
print(answer)



Hi, Welcome to Chat Doctor forum.The common signs and symptoms of having a blood clot in the lung (pulmonary embolism) include


In [ ]:
# Example search
query = "HI doctor, What are common symptoms of flu?"
answer = retrieve_answer_bow(query, train_dataset, vectorizer, X)
print(answer)



Hello. According to your statement, you got a fever that subsided after two days and then developed flu and mouth sore. Mouth sore is a common complication of flu. Again mouth sore may develope by a viral infection like herpes simplex virus most commonly fungal infection, nutritional deficiency like iron, zinc, vitamin B12, etc. Some medications like corticosteroid, chemotherapy, sulfa  ChatDoctor.  Autoimmune disorders like lichen planus also. For your flu-like running nose and body ache you can take anti-histamine, Acetamenofen or Paracetamol, topical antiviral cream, vitamins containing B12 with zinc and iron. If any complications occur or your symptoms do not subside within seven to ten days inform me as soon as possible, I will try to help you further.


In [ ]:
# Example search
query= "Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (oral). Now, I am feeling the pain frequently in the right lower abdomen with mild bleeding and pain. It is less when I lie on bed. What to do?"
answer = retrieve_answer_bow(query, train_dataset, vectorizer, X)
print(answer)



Hello, Welcome to Chat Doctor forum. I have read through your question and understand your concerns. I think it is threatened abortion. You should take bed rest along with tablet Susten 200 mg twice daily (Progesterone) for two weeks and Mbryosafe sachet once daily (L-arginine). I think it will help you and give you better prognosis. Ultrasound is required after two weeks for evaluation.


In [ ]:
# Example search
query="Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigmentation caused by acne before. I only get acne on my cheeks. What may be the reason? I usually get cyst acne bumps on my cheeks. If acne clears, then it comes again. I have red pigmentation caused by acne. Can you recommend a product for acne and after acne has cleared, can you suggest a product for pigmentation? Can you recommend a product for controlling further breakouts? Also, I wanted to know which sunscreen and moisturizer to use?"
answer = retrieve_answer_bow(query, train_dataset, vectorizer, X)
print(answer)



Hi. Acne reduces in severity with aging. Use cleanser like Cetaphil to wash face. Apply Retino-A (Retinol) cream in the night, Clindamycin gel in the morning. Treat dan ChatDoctor.  You can use sunscreens like Shade (Avobenzone, Oxybenzone) lotion. Retino-A helps in pigmentation and to some extent scars. You can consider lasers later if scars persist. For more information consult a dermatologist online


In [ ]:
# Example search
query="Hello doctor,I am diabetic and hypertensive and have neuro problems. I want you to look at my MRI reports and advice me accordingly. I have attached the reports."
answer = retrieve_answer_bow(query, train_dataset, vectorizer, X)
print(answer)


Hello, Welcome to Chat Doctor forum. I have gone through the attachments (attachment removed to protect patient identity). The MRI (magnetic resonance imaging) report says that there are chronic ischemic changes suggestive of small vessel ischemia. These changes are usually seen in people with underlying diabetes and hypertension and represent longstanding nature of two conditions. The only way to prevent further progression of the changes is strict control of diabetes and hypertension. These changes in the brain will not lead to any disability. However, you should understand that you are at increased risk of stroke and hence strict control of diabetes and hypertension is essential. For more information consult a neurosurgeon online


In [ ]:
#------------------------------------------------
# Semantic Search (Using Embeddings without RAG)
#------------------------------------------------


In [ ]:
# Initialize HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

# Create FAISS index from Document objects
db_emb = FAISS.from_documents(documents, embedding_model)

# Initialize Retriever
retriever = db_emb.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

# Function to retrieve answers using Semantic Search
def retrieve_answer_semantic(query, retriever, top_k=3):
    retrieved_docs = retriever.get_relevant_documents(query)
    if not retrieved_docs:
        return "No relevant answers found."
    # For simplicity, return the top 1 answer
    return retrieved_docs[0].metadata['answer']




In [ ]:
# Example search
query = "Hi doctor, What are the symptoms of diabetes?"
answer = retrieve_answer_semantic(query, retriever)
print(answer)


Hello. I have seen your attached details (attachment removed to protect patient identity), which show you are on multiple  ChatDoctor.  However, if levels


In [ ]:
# Example search
query = "HI doctor, What are common symptoms of flu?"
answer = retrieve_answer_semantic(query, retriever)
print(answer)


Hello, Welcome to Chat Doctor forum. I have gone through your query, and I can understand your concerns. The symptoms you have mentioned are suggestive of pharyngitis (a sore throat) along with a common cold. You do not need to worry regarding this as it is highly curable and complications are rare. I suggest you consult a general practitioner and get the following things done. For more information consult a general practitioner online


In [ ]:
# Example search
query= "Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (oral). Now, I am feeling the pain frequently in the right lower abdomen with mild bleeding and pain. It is less when I lie on bed. What to do?"
answer = retrieve_answer_semantic(query, retriever)
print(answer)


Hello, Welcome to Chat Doctor forum. I have read through your question and understand your concerns. I think it is threatened abortion. You should take bed rest along with tablet Susten 200 mg twice daily (Progesterone) for two weeks and Mbryosafe sachet once daily (L-arginine). I think it will help you and give you better prognosis. Ultrasound is required after two weeks for evaluation.


In [ ]:
# Example search
query="Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigmentation caused by acne before. I only get acne on my cheeks. What may be the reason? I usually get cyst acne bumps on my cheeks. If acne clears, then it comes again. I have red pigmentation caused by acne. Can you recommend a product for acne and after acne has cleared, can you suggest a product for pigmentation? Can you recommend a product for controlling further breakouts? Also, I wanted to know which sunscreen and moisturizer to use?"
answer = retrieve_answer_semantic(query, retriever)
print(answer)


Hi. Acne reduces in severity with aging. Use cleanser like Cetaphil to wash face. Apply Retino-A (Retinol) cream in the night, Clindamycin gel in the morning. Treat dan ChatDoctor.  You can use sunscreens like Shade (Avobenzone, Oxybenzone) lotion. Retino-A helps in pigmentation and to some extent scars. You can consider lasers later if scars persist. For more information consult a dermatologist online


In [ ]:
# Example search
query="Hello doctor,I am diabetic and hypertensive and have neuro problems. I want you to look at my MRI reports and advice me accordingly. I have attached the reports."
answer = retrieve_answer_semantic(query, retriever)
print(answer)


Hello, Welcome to Chat Doctor forum. I have gone through the attachments (attachment removed to protect patient identity). The MRI (magnetic resonance imaging) report says that there are chronic ischemic changes suggestive of small vessel ischemia. These changes are usually seen in people with underlying diabetes and hypertension and represent longstanding nature of two conditions. The only way to prevent further progression of the changes is strict control of diabetes and hypertension. These changes in the brain will not lead to any disability. However, you should understand that you are at increased risk of stroke and hence strict control of diabetes and hypertension is essential. For more information consult a neurosurgeon online


In [ ]:
#---------------
# RAG TIME
#---------------


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(documents)

print("Dataset Splited by Chunks - You have {0} number of chunks.".format(len(chunked_docs)))


Dataset Splited by Chunks - You have 10343 number of chunks.


In [ ]:
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2'))

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

model_name = "HuggingFaceH4/zephyr-7b-beta"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# Pipeline for text generation
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Prompt template to match desired output format
prompt_template = """
=================================================================================================
You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
=================================================================================================
Context:
{context}
=================================================================================================
Question: {question}
=================================================================================================
Answer:
=================================================================================================
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


Device set to use cuda:0


In [ ]:
question = "Hi doctor, What are the symptoms of diabetes?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)



You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
Context:
[Document(id='272de830-d0c9-478a-9178-7e1b96486480', metadata={'answer': 'Hello. I have seen your attached details (attachment removed to protect patient identity), which show you are on multiple  ChatDoctor.  However, if levels'}, page_content='Hello doctor,I am 20 years old, type 2 diabetes patient. Please give me information about diabetes. Every morning, I do running around 2 to 3 km.'), Document(id='d0b419c4-412a-4bec-8579-ded01cda26e8', metadata={'answer': 'Hello, Welcome to Chat Doctor forum. I can understand your concern. According to your history you have been suffering from frequent urination and feeling extremely thirsty especially during night after dinner and before 

In [ ]:
question = "HI doctor, What are common symptoms of flu?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)



You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
Context:
[Document(id='3cb302fd-fab9-4f9a-b1b8-c74fd96022e7', metadata={'answer': 'Hi. I have read your query and understood your health concern. Infection by Streptococcus or viral. 1. Complete blood count.2. Chest x-ray.3. Nasal and throat swab culture.4. PCR.  Take care.'}, page_content='now. I am wondering are stomach cramps, chest pain, back pain, and labored breathing symptoms of flu? Please do the needful.'), Document(id='06ccc003-02aa-4d4f-927a-bb70f5cbabc4', metadata={'answer': 'Hello, Welcome to Chat Doctor forum. I have gone through your query, and I can understand your concerns. The symptoms you have mentioned are suggestive of pharyngitis (a sore throat) along with a common c

In [ ]:
question = "Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (oral). Now, I am feeling the pain frequently in the right lower abdomen with mild bleeding and pain. It is less when I lie on bed. What to do?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)



You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
Context:
[Document(id='a060ccfc-9ed8-48ad-8070-4a13ea6b1753', metadata={'answer': 'Hello, Welcome to Chat Doctor forum. I have read through your question and understand your concerns. I think it is threatened abortion. You should take bed rest along with tablet Susten 200 mg twice daily (Progesterone) for two weeks and Mbryosafe sachet once daily (L-arginine). I think it will help you and give you better prognosis. Ultrasound is required after two weeks for evaluation.'}, page_content='Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (or

In [ ]:
question = "Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigmentation caused by acne before. I only get acne on my cheeks. What may be the reason? I usually get cyst acne bumps on my cheeks. If acne clears, then it comes again. I have red pigmentation caused by acne. Can you recommend a product for acne and after acne has cleared, can you suggest a product for pigmentation? Can you recommend a product for controlling further breakouts? Also, I wanted to know which sunscreen and moisturizer to use?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)



You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
Context:
[Document(id='4e4fe570-031e-4fba-9339-df363e7f9110', metadata={'answer': 'Hi. Acne reduces in severity with aging. Use cleanser like Cetaphil to wash face. Apply Retino-A (Retinol) cream in the night, Clindamycin gel in the morning. Treat dan ChatDoctor.  You can use sunscreens like Shade (Avobenzone, Oxybenzone) lotion. Retino-A helps in pigmentation and to some extent scars. You can consider lasers later if scars persist. For more information consult a dermatologist online'}, page_content='Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigment

In [ ]:
question = "Hello doctor,I am diabetic and hypertensive and have neuro problems. I want you to look at my MRI reports and advice me accordingly. I have attached the reports."
# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)



You are an expert doctor giving a precise answer based on the provided context.
If you don't find the response in the context, just say "I haven't found the answer".
If you don't know the reply, just say "I don't know", don't try to make up an answer.
Give the return in bullet points within 50 words.
Context:
[Document(id='e617e261-ea8a-442f-a902-21a7b75a2921', metadata={'answer': 'Hello, Welcome to Chat Doctor forum. I have gone through the attachments (attachment removed to protect patient identity). The MRI (magnetic resonance imaging) report says that there are chronic ischemic changes suggestive of small vessel ischemia. These changes are usually seen in people with underlying diabetes and hypertension and represent longstanding nature of two conditions. The only way to prevent further progression of the changes is strict control of diabetes and hypertension. These changes in the brain will not lead to any disability. However, you should understand that you are at increased risk 

In [ ]:
#============================
# Agent
#============================

In [ ]:
# Pipeline for text generation
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)



Device set to use cuda:0


In [ ]:
medical_assistant = autogen.AssistantAgent(
    name="medical_assistant",
    system_message="You are a medical assistant involved in the analysis of patients disease.",
    llm_config=llm_config,
)

In [ ]:
task= '''
      Firstly review the {result} from "rag_chain.invoke(question)" and refine {result} adding details from [metadata].
      Then write a concise medical recommendation to user's question in bullet points.
      Please write both answers within 50 words for each one.
      '''

In [ ]:
question = "Hi doctor, What are the symptoms of diabetes?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)
# Prepare the messages for the assistant
messages = [
        {"role": "user", "content": question},
        {"role": "user", "content": f"Here is the retrieved context: {result}"},
        {"role": "user", "content": task}
]
# Generate the reply using the assistant
reply = medical_assistant.generate_reply(messages=messages)

# Display the output
print('=======================================================================================')
print("Agent Review with Medical Recommendation:")
print('=======================================================================================')
print(reply)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Agent Review with Medical Recommendation:
**Refined Result:**
- Increased thirst and frequent urination, especially at night.
- Unexplained weight loss or gain.
- Fatigue and weakness.
- Blurred vision.
- Slow healing of cuts and wounds.
- Tingling or numbness in hands or feet.
- Recurrent infections.

**Medical Recommendation:**
- Monitor blood sugar levels regularly.
- Consult a healthcare professional for testing (HbA1c).
- Maintain a balanced diet and exercise regimen.
- Stay hydrated and manage stress.
- Schedule regular check-ups for diabetes management.


In [ ]:
question = "HI doctor, What are common symptoms of flu?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)
# Prepare the messages for the assistant
messages = [
        {"role": "user", "content": question},
        {"role": "user", "content": f"Here is the retrieved context: {result}"},
        {"role": "user", "content": task}
]
# Generate the reply using the assistant
reply = medical_assistant.generate_reply(messages=messages)

# Display the output
print('=======================================================================================')
print("Agent Review with Medical Recommendation:")
print('=======================================================================================')
print(reply)


Agent Review with Medical Recommendation:
**Refined Result:**
Common symptoms of flu include:
- Fever
- Cough
- Fatigue
- Body aches
- Headaches
- Sometimes diarrhea and vomiting
Symptoms typically appear suddenly and may last from several days to two weeks. It's crucial to consult a healthcare provider if symptoms persist or worsen.

**Medical Recommendation:**
- Stay hydrated and rest.
- Avoid contact with others to prevent spreading the virus.
- Monitor symptoms closely, and seek medical attention if they worsen or persist.


In [ ]:
question = "Hi doctor,My last USG report showed intrauterine pregnancy with 8 mm gestation sac, no fetal and yolk sac. Last week I had brown discharge with mild lower abdominal pain and was prescribed Susten 200 mg (oral). Now, I am feeling the pain frequently in the right lower abdomen with mild bleeding and pain. It is less when I lie on bed. What to do?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)
# Prepare the messages for the assistant
messages = [
        {"role": "user", "content": question},
        {"role": "user", "content": f"Here is the retrieved context: {result}"},
        {"role": "user", "content": task}
]
# Generate the reply using the assistant
reply = medical_assistant.generate_reply(messages=messages)

# Display the output
print('=======================================================================================')
print("Agent Review with Medical Recommendation:")
print('=======================================================================================')
print(reply)


Agent Review with Medical Recommendation:
### Reviewed Result:
- Continue Susten 200 mg twice daily for progesterone support.
- Consider adding Mbryosafe sachet (L-arginine) daily.
- Take complete bed rest to relieve pain.
- Follow up with an ultrasound in two weeks for evaluation.
- Seek immediate consultation if symptoms worsen or new issues arise.

### Medical Recommendation:
- Adhere to prescribed medications (Susten and Mbryosafe).
- Prioritize rest to manage abdominal pain.
- Monitor for worsening symptoms or increased bleeding.
- Plan an ultrasound in two weeks to check pregnancy status.
- Contact your doctor for urgent issues.


In [ ]:
question = "Hello doctor,I am currently having acne on my cheeks. I am currently applying Benzoyl peroxide gel on my acne. It only reduces the inflammation but bumps are not reducing. I also have pigmentation caused by acne before. I only get acne on my cheeks. What may be the reason? I usually get cyst acne bumps on my cheeks. If acne clears, then it comes again. I have red pigmentation caused by acne. Can you recommend a product for acne and after acne has cleared, can you suggest a product for pigmentation? Can you recommend a product for controlling further breakouts? Also, I wanted to know which sunscreen and moisturizer to use?"
# Invoke the chain to generate answers
result = rag_chain.invoke(question)
# Prepare the messages for the assistant
messages = [
        {"role": "user", "content": question},
        {"role": "user", "content": f"Here is the retrieved context: {result}"},
        {"role": "user", "content": task}
]
# Generate the reply using the assistant
reply = medical_assistant.generate_reply(messages=messages)

# Display the output
print('=======================================================================================')
print("Agent Review with Medical Recommendation:")
print('=======================================================================================')
print(reply)


Agent Review with Medical Recommendation:
**Review of Result:**
- Apply Cetaphil cleanser to wash face.
- Use Retino-A (Retinol) cream at night and Clindamycin gel in the morning for acne.
- For pigmentation, consider vitamin C or niacinamide products.
- Use broad-spectrum sunscreen (SPF 30+) and light, oil-free moisturizer.

**Concise Medical Recommendation:**
- Cleanse with Cetaphil; treat acne using Retino-A at night and Clindamycin during the day.
- After clearing, use vitamin C or niacinamide for pigmentation.
- Apply broad-spectrum sunscreen (SPF 30+) and lightweight, oil-free moisturizer daily.


In [ ]:
question = "Hello doctor,I am diabetic and hypertensive and have neuro problems. I want you to look at my MRI reports and advice me accordingly. I have attached the reports."
# Invoke the chain to generate answers
result = rag_chain.invoke(question)
# Prepare the messages for the assistant
messages = [
        {"role": "user", "content": question},
        {"role": "user", "content": f"Here is the retrieved context: {result}"},
        {"role": "user", "content": task}
]
# Generate the reply using the assistant
reply = medical_assistant.generate_reply(messages=messages)

# Display the output
print('=======================================================================================')
print("Agent Review with Medical Recommendation:")
print('=======================================================================================')
print(reply)


Agent Review with Medical Recommendation:
**Refined Result:**
- The MRI report shows chronic ischemic changes indicating small vessel ischemia, often associated with diabetes and hypertension.
- To prevent progression, it's crucial to manage diabetes and hypertension strictly.
- Although these changes won't cause disability, they increase stroke risk. Consulting a neurosurgeon is advised.

**Medical Recommendation:**
- Maintain strict control of blood sugar and blood pressure levels.
- Follow up regularly with healthcare providers to monitor conditions.
- Consider lifestyle changes, such as diet and exercise, for better management.
- Seek immediate medical attention for any new neurological symptoms.
